# Showcase

This paper uses more complicated architecture for solving ***Flexible** Job-Shop Scheduling Problem*.  This paper is a bit more technical and complicate, so I didn't dive into the details. I just tried to make the model work. There were 3 checkpoints in the official repository, but I managed to run only one of them.

## Preprocessing

Since this model works on **FJSP**, I need to trasform our JSSP benchmarks to FJSP as in the model [fjsp-drl](/models/fjsp-drl/repo/Showcase%20fjsp-drl.ipynb). I will just reuse the same code.

In [4]:
import os
import argparse
import numpy as np

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../../benchmarks/jssp"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def jssp_taillard_to_fjsp(filename: str) -> str:
    '''Transforms JSSP instance in Taillard's specification to FJSP instance
       and stores it in a temporary file
    
      Args:
        filename - name of the file with JSSP instance in Taillard's specification
        
      Returns:
        string - filename of the equivalent FJSP instance 
    '''
    # parse JSSP Taillard instance
    J, M, processor_times, orders_of_machines = parse_instance_taillard(filename)
    
    # convert JSSP to FJSP
    with open("/tmp/fjsp_" + filename.split("/")[-1], 'w') as f:
        # write number of jobs, number of machines, and jobs/machines (which is always 1 for JSSP)
        f.write(str(J) + "   " + str(M) + "   1\n")
        
        # each line is a job
        for i in range(J):
            # each line starts with the number of operations in a job
            number_of_operations = len(processor_times[i])
            f.write(str(number_of_operations) + "  ")
            
            # print the operation as a tuple (number of available machines, current machine, processing time)
            for j in range(number_of_operations):
                f.write(" 1   " + str(orders_of_machines[i][j]) + "   " + str(processor_times[i][j]) + "  ")
                
            f.write('\n')
            
    return "/tmp/fjsp_" + filename.split("/")[-1]

The original code is a bit messy, so I had to make a small wrapper around the original code to make it nicer.

In [5]:
import time
from Params import configs
from validation_realWorld import test

def solve_fjsp_instance(model, instance):
    '''Solves FJSP instance using given model

    Args:
      model - model to use for solving the instance
      instance - instance to be solved

    Returns:
      makespan of the instance,
      time it took to solve the instance
    '''
    # get number of machines
    with open(instance, 'r') as f:
        M = int(f.readline().strip().split()[1])

    # override the value of number of machines in the configs according to the instance 
    # this is not handled in the original code and causes errors
    setattr(configs, 'n_m', M)

    # solve the instance
    start = time.time()
    makespan = test(model, instance)
    end = time.time()

    return makespan, end - start

## Run the model on instances from the paper available in the official repository

Only subset of instances from the paper are available in the official repository.

In [6]:
from validation_realWorld import get_imlist

MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'
INSTANCES_PATH = './FJSSPinstances/M15'

for instance in sorted(get_imlist(INSTANCES_PATH)):
    makespan, run_time = solve_fjsp_instance(MODEL_PATH, instance)
    print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")

/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/End-to-end-DRL-for-FJSP/repo/FJSP_RealWorld/mb_agg.py:44: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  graph_pool = torch.sparse.FloatTensor(idx, elem,
/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/End-to-end-DRL-for-FJSP/repo/FJSP_RealWorld/FJSP_Env.py:221: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  self.adj = torch.tensor(self.adj)


Instance: HurinkVdata39.fjs, makespan: 972.0, time: 0.75
Instance: HurinkVdata40.fjs, makespan: 1030.0, time: 0.56
Instance: HurinkVdata41.fjs, makespan: 960.0, time: 0.56
Instance: HurinkVdata42.fjs, makespan: 988.0, time: 0.56
Instance: HurinkVdata43.fjs, makespan: 974.0, time: 0.55
Instance: HurinkVdata46.fjs, makespan: 524.0, time: 0.84
Instance: HurinkVdata47.fjs, makespan: 560.5, time: 0.82
Instance: HurinkVdata48.fjs, makespan: 555.0, time: 0.83


## Run the model on JSSP benchmarks

In [4]:
MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'

exceptions = [
    '../../../../benchmarks/jssp/orb_instances/Taillard_specification/orb07.txt' # this instance gets stuck while solving, I haven't figured out why
]

for instance in sorted(get_all_instances_in_taillard_specification()):
    if instance in exceptions:
        print('Skipping', instance)
        continue

    makespan, run_time = solve_fjsp_instance(MODEL_PATH, jssp_taillard_to_fjsp(instance))
    print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")

Instance: abz5.txt, makespan: 1382.0, time: 0.21
Instance: abz6.txt, makespan: 1194.0, time: 0.21
Instance: abz7.txt, makespan: 780.0, time: 0.84
Instance: abz8.txt, makespan: 905.0, time: 0.85
Instance: abz9.txt, makespan: 913.0, time: 0.84
Instance: dmu01.txt, makespan: 3391.0, time: 0.85
Instance: dmu02.txt, makespan: 3561.0, time: 0.85
Instance: dmu03.txt, makespan: 3427.0, time: 0.85
Instance: dmu04.txt, makespan: 3395.0, time: 0.84
Instance: dmu05.txt, makespan: 3490.0, time: 0.85
Instance: dmu06.txt, makespan: 4276.0, time: 1.31
Instance: dmu07.txt, makespan: 3914.0, time: 1.3
Instance: dmu08.txt, makespan: 3879.0, time: 1.28
Instance: dmu09.txt, makespan: 4250.0, time: 1.33
Instance: dmu10.txt, makespan: 3859.0, time: 1.3
Instance: dmu11.txt, makespan: 4381.0, time: 1.55
Instance: dmu12.txt, makespan: 4587.0, time: 1.61
Instance: dmu13.txt, makespan: 4995.0, time: 1.59
Instance: dmu14.txt, makespan: 4212.0, time: 1.6
Instance: dmu15.txt, makespan: 4222.0, time: 1.56
Instance: d

# Run the model on FJSP benchmarks

In [7]:
def get_all_fjsp_instances():
    '''Lists all FJSP instances'''
    matching_files = []
    root_dir = "../../../../benchmarks/fjsp"
    target_string = ".fjs"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [10]:
MODEL_PATH = 'saved_network/FJSP_J15M15/best_value0'

for instance in sorted(get_all_fjsp_instances()):
    try:
        makespan, run_time = solve_fjsp_instance(MODEL_PATH, instance)
        print(f"Instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")
    except Exception as e:
        print(f"Failed to solve instance {instance}, error: {e}")

Instance: Behnke1.fjs, makespan: 110.0, time: 0.14
Instance: Behnke10.fjs, makespan: 163.5, time: 0.25
Instance: Behnke11.fjs, makespan: 286.0, time: 1.1
Instance: Behnke12.fjs, makespan: 276.0, time: 1.15
Instance: Behnke13.fjs, makespan: 301.0, time: 1.06
Instance: Behnke14.fjs, makespan: 300.5, time: 1.08
Instance: Behnke15.fjs, makespan: 286.0, time: 1.03
Instance: Behnke16.fjs, makespan: 508.5, time: 4.86
Instance: Behnke17.fjs, makespan: 486.5, time: 4.8
Instance: Behnke18.fjs, makespan: 501.5, time: 4.81
Instance: Behnke19.fjs, makespan: 495.0, time: 4.94
Instance: Behnke2.fjs, makespan: 110.0, time: 0.11
Instance: Behnke20.fjs, makespan: 491.5, time: 5.05
Instance: Behnke21.fjs, makespan: 105.0, time: 0.13
Instance: Behnke22.fjs, makespan: 107.0, time: 0.16
Instance: Behnke23.fjs, makespan: 106.0, time: 0.13
Instance: Behnke24.fjs, makespan: 108.0, time: 0.13
Instance: Behnke25.fjs, makespan: 96.0, time: 0.13
Instance: Behnke26.fjs, makespan: 150.0, time: 0.29
Instance: Behnke2